In [1]:
feedback_produto = """
Estou muito satisfeito com o Smartphone XYZ Pro. O desempenho é excelente, e o sistema
operacional é rápido e intuitivo. A câmera é um dos principais destaques, especialmente o
modo noturno, que captura imagens incríveis mesmo em baixa iluminação. A duração da bateria
também impressiona, durando facilmente um dia inteiro com uso intenso.
Por outro lado, sinto que o produto poderia ser melhorado em alguns aspectos. A tela,
embora tenha cores vibrantes, parece refletir bastante luz, dificultando o uso sob o sol.
Além disso, o carregador incluído na caixa não oferece carregamento rápido, o que é um ponto
negativo considerando o preço do aparelho
"""


In [2]:
from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template("""
Para o texto a seguir, extraia as seguintes informações:
produto: Nome do produto mencionado no texto.
caracteristicas_positivas: Liste todas as características positivas mencionadas sobre o produto.
caracteristicas_negativas: Liste todas as características negativas mencionadas sobre o produto.
recomendacao: O cliente recomenda esse produto? Responda True para sim e False para não.

Texto: {review}

Retorne a resposta no formato JSON
"""
)



In [3]:
print(review_template.format_messages(review=feedback_produto))

[HumanMessage(content='\nPara o texto a seguir, extraia as seguintes informações:\nproduto: Nome do produto mencionado no texto.\ncaracteristicas_positivas: Liste todas as características positivas mencionadas sobre o produto.\ncaracteristicas_negativas: Liste todas as características negativas mencionadas sobre o produto.\nrecomendacao: O cliente recomenda esse produto? Responda True para sim e False para não.\n\nTexto: \nEstou muito satisfeito com o Smartphone XYZ Pro. O desempenho é excelente, e o sistema\noperacional é rápido e intuitivo. A câmera é um dos principais destaques, especialmente o\nmodo noturno, que captura imagens incríveis mesmo em baixa iluminação. A duração da bateria\ntambém impressiona, durando facilmente um dia inteiro com uso intenso.\nPor outro lado, sinto que o produto poderia ser melhorado em alguns aspectos. A tela,\nembora tenha cores vibrantes, parece refletir bastante luz, dificultando o uso sob o sol.\nAlém disso, o carregador incluído na caixa não ofer

In [4]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()
resposta = chat.invoke(review_template.format_messages(review=feedback_produto))

In [6]:
resposta.content

'{\n    "produto": "Smartphone XYZ Pro",\n    "caracteristicas_positivas": [\n        "Desempenho excelente",\n        "Sistema operacional rápido e intuitivo",\n        "Câmera de destaque, especialmente o modo noturno",\n        "Duração da bateria impressionante"\n    ],\n    "caracteristicas_negativas": [\n        "Tela reflete bastante luz",\n        "Carregador não oferece carregamento rápido"\n    ],\n    "recomendacao": true\n}'

In [7]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [8]:
schema_produto = ResponseSchema(
    name="produto",
    type="string",
    description="Nome do produto mencionado no texto."
)

schema_positivas = ResponseSchema(
    name="caracteristicas_positivas",
    type="list",
    description="Lista de características positivas mencionadas sobre o produto."
)

schema_negativas = ResponseSchema(
    name="caracteristicas_negativas",
    type="list",
    description="Lista de características negativas mencionadas sobre o produto.",
)

schema_recomendacao = ResponseSchema(
    name="recomendacao",
    type="bool",
    description="O cliente recomenda esse produto? Responda True para sim e False para não.",
)



In [9]:
response_schema = [schema_produto, schema_positivas, schema_negativas, schema_recomendacao]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
schema_formatado = output_parser.get_format_instructions()

In [10]:
print(schema_formatado)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"produto": string  // Nome do produto mencionado no texto.
	"caracteristicas_positivas": list  // Lista de características positivas mencionadas sobre o produto.
	"caracteristicas_negativas": list  // Lista de características negativas mencionadas sobre o produto.
	"recomendacao": bool  // O cliente recomenda esse produto? Responda True para sim e False para não.
}
```


In [11]:
review_template2 = ChatPromptTemplate.from_template("""
Para o texto a seguir extraia as seguintes informações:
produto, caracteristicas_positivas, caracteristicas_negativas e recomendacao.

Texto: {review}

{schema}
""", partial_variables={"schema": schema_formatado}
)

In [12]:
print(review_template2.format_messages(review=feedback_produto))

[HumanMessage(content='\nPara o texto a seguir extraia as seguintes informações:\nproduto, caracteristicas_positivas, caracteristicas_negativas e recomendacao.\n\nTexto: \nEstou muito satisfeito com o Smartphone XYZ Pro. O desempenho é excelente, e o sistema\noperacional é rápido e intuitivo. A câmera é um dos principais destaques, especialmente o\nmodo noturno, que captura imagens incríveis mesmo em baixa iluminação. A duração da bateria\ntambém impressiona, durando facilmente um dia inteiro com uso intenso.\nPor outro lado, sinto que o produto poderia ser melhorado em alguns aspectos. A tela,\nembora tenha cores vibrantes, parece refletir bastante luz, dificultando o uso sob o sol.\nAlém disso, o carregador incluído na caixa não oferece carregamento rápido, o que é um ponto\nnegativo considerando o preço do aparelho\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"produto": s

In [ ]:
resposta = chat.invoke(review_template2.format_messages(review=feedback_produto))
resposta_json = output_parser.parse(resposta.content)